In [1]:
import os
import sys

import warnings as warnings
warnings.filterwarnings(action='ignore')

import pandas as pd

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', -1)
pd.options.display.float_format = '{:.2f}'.format # 소수점 유효숫자 표기법으로 안쓰기

In [2]:
data_folder = 'data/'

In [3]:
%%time
# train_df = pd.read_csv(os.path.join(data_folder,'train.csv'))
train_ability_df = pd.read_csv(os.path.join(data_folder,'train_Ability.csv'))

CPU times: user 2.41 s, sys: 297 ms, total: 2.7 s
Wall time: 4.34 s


In [26]:
train_df[(train_df['game_id'] == 30000) & (train_ability_df['player'] == 1)]

,game_id,winner,time,player,species,event,event_contents


In [ ]:
유닛 생산
건물 건설
업그레이드
스킬 사용
리페어

공격
홀드
캔슬
스탑

In [28]:
train_ability_df = train_ability_df[(train_ability_df['game_id'] == 0) & (train_ability_df['player'] == 1)]

In [36]:
aaa.head()

,game_id,winner,time,player,species,event,event_contents
3,0,1,0.33,1,T,Ability,(1360) - TrainSCV
5,0,1,0.37,1,T,Ability,"(1021) - BuildSupplyDepot; Location: (28.0, 148.0, 49127)"
6,0,1,0.40,1,T,Ability,"(1022) - BuildRefinery; Target: CreepOnlyBlocker4x4 [024C0001]; Location: (27.5, 154.5, 49120)"
7,0,1,0.41,1,T,Ability,(1360) - TrainSCV
11,0,1,1.21,1,T,Ability,"(1023) - BuildBarracks; Location: (28.5, 144.5, 49127)"


In [40]:
# %%time
# terran_ability_df = train_ability_df[(train_ability_df['species'] == 'T')]
# terran_ability_df.to_csv(os.path.join(data_folder,'terran_ability.csv'),index=False)

# protoss_ability_df = train_ability_df[(train_ability_df['species'] == 'P')]
# protoss_ability_df.to_csv(os.path.join(data_folder,'protoss_ability.csv'),index=False)

# zerg_ability_df = train_ability_df[(train_ability_df['species'] == 'Z')]
# zerg_ability_df.to_csv(os.path.join(data_folder,'zerg_ability.csv'),index=False)

CPU times: user 12.7 s, sys: 433 ms, total: 13.1 s
Wall time: 15 s


In [41]:
terran_ability_df = pd.read_csv(os.path.join(data_folder,'terran_ability.csv'))
protoss_ability_df = pd.read_csv(os.path.join(data_folder,'protoss_ability.csv'))
zerg_ability_df = pd.read_csv(os.path.join(data_folder,'zerg_ability.csv'))

In [43]:
print(terran_ability_df.shape)
print(protoss_ability_df.shape)
print(zerg_ability_df.shape)

(1717228, 7)
(923530, 7)
(1157632, 7)


In [49]:
terran_ability_list = sorted(terran_ability_df.event_contents.map(lambda x: x.split(';')[0]).unique())
protoss_ability_list = sorted(protoss_ability_df.event_contents.map(lambda x: x.split(';')[0]).unique())
zerg_ability_list = sorted(zerg_ability_df.event_contents.map(lambda x: x.split(';')[0]).unique())

In [54]:
terran_ability_list

['(1000) - SCVRepair',
 '(1020) - BuildCommandCenter',
 '(1021) - BuildSupplyDepot',
 '(1022) - BuildRefinery',
 '(1023) - BuildBarracks',
 '(1024) - BuildEngineeringBay',
 '(1025) - BuildMissileTurret',
 '(1026) - BuildBunker',
 '(1028) - BuildSensorTower',
 '(1029) - BuildGhostAcademy',
 '(102A) - BuildFactory',
 '(102B) - BuildStarport',
 '(102D) - BuildArmory',
 '(102F) - BuildFusionCore',
 '(103E) - CancelTerranBuilding',
 '(1060) - UseStimpack',
 '(1080) - CloakGhost',
 '(1081) - DecloakGhost',
 '(10C0) - HealMedivac',
 '(10E0) - SiegeMode',
 '(1100) - TankMode',
 '(1120) - CloakBanshee',
 '(1121) - DecloakBanshee',
 '(1140) - LoadTargetMedivac',
 '(1142) - UnloadTargetMedivac',
 '(1143) - UnloadUnitMedivac',
 '(1160) - ScannerSweep',
 '(1180) - YamatoGun',
 '(11A0) - AssaultMode',
 '(11C0) - FighterMode',
 '(11E0) - LoadTargetBunker',
 '(11E1) - UnloadAllBunker',
 '(11E3) - UnloadUnitBunker',
 '(1201) - UnloadAllCommandCenter',
 '(1203) - UnloadUnitCommandCenter',
 '(1204) - Loa

In [55]:
protoss_ability_list

['(1540) - BuildNexus',
 '(1541) - BuildPylon',
 '(1542) - BuildAssimilator',
 '(1543) - BuildGateway',
 '(1544) - BuildForge',
 '(1545) - BuildFleetBeacon',
 '(1546) - BuildTwilightCouncil',
 '(1547) - BuildPhotonCannon',
 '(1549) - BuildStargate',
 '(154A) - BuildTemplarArchive',
 '(154B) - BuildDarkShrine',
 '(154C) - BuildRoboticsBay',
 '(154D) - BuildRoboticsFacility',
 '(154E) - BuildCyberneticsCore',
 '(1560) - LoadTarget',
 '(1562) - UnloadTargetWarpPrism',
 '(1563) - UnloadUnitWarpPrism',
 '(1580) - TrainZealot',
 '(1581) - TrainStalker',
 '(1583) - TrainHighTemplar',
 '(1584) - TrainDarkTemplar',
 '(1585) - TrainSentry',
 '(1586) - TrainAdept',
 '(15A0) - TrainPhoenix',
 '(15A2) - TrainCarrier',
 '(15A4) - TrainVoidRay',
 '(15A8) - TrainOracle',
 '(15A9) - TrainTempest',
 '(15C0) - TrainWarpPrism',
 '(15C1) - TrainObserver',
 '(15C2) - TrainColossus',
 '(15C3) - TrainImmortal',
 '(15D2) - TrainDisruptor',
 '(15E0) - TrainProbe',
 '(1600) - PsionicStorm',
 '(1660) - TrainInter

In [56]:
zerg_ability_list

['(16E0) - BuildHatchery',
 '(16E2) - BuildExtractor',
 '(16E3) - BuildSpawningPool',
 '(16E4) - BuildEvolutionChamber',
 '(16E5) - BuildHydraliskDen',
 '(16E6) - BuildSpire',
 '(16E7) - BuildUltraliskCavern',
 '(16E8) - BuildInfestationPit',
 '(16E9) - BuildNydusNetwork',
 '(16EA) - BuildBanelingNest',
 '(16ED) - BuildRoachWarren',
 '(16EE) - BuildSpineCrawler',
 '(16EF) - BuildSporeCrawler',
 '(1700) - Gather',
 '(1701) - ReturnCargo',
 '(1720) - ResearchZergMeleeWeaponsLevel1',
 '(1721) - ResearchZergMeleeWeaponsLevel2',
 '(1723) - ResearchZergGroundArmorsLevel1',
 '(1724) - ResearchZergGroundArmorsLevel2',
 '(1726) - ResearchZergMissileWeaponsLevel1',
 '(1727) - ResearchZergMissileWeaponsLevel2',
 '(1740) - UpgradeToLair',
 '(1741) - CancelUpgradeToLair',
 '(1760) - UpgradeToHive',
 '(1761) - CancelUpgradeToHive',
 '(1780) - MorphToGreaterSpire',
 '(17A1) - EvolvePneumatizedCarapace',
 '(17A3) - EvolveBurrow',
 '(17C0) - EvolveAdrenalGlands',
 '(17C1) - EvolveMetabolicBoost',
 '(17

인구수 계산 로직을 만들어봐야겠다
- 인구수 건물로
- 유닛으로 
- (취소 고려)
- 파괴되거나 죽은건 고려 안함

In [89]:
ability_location = train_ability_df.event_contents.map(lambda x: x.split(';')).map(lambda x: x[-1].split(': ')[-1] if len(x) > 1 else None)
# protoss_ability_list = sorted(protoss_ability_df.event_contents.map(lambda x: x.split(';')[0]).unique())
# zerg_ability_list = sorted(zerg_ability_df.event_contents.map(lambda x: x.split(';')[0]).unique())

ability_location_df = pd.DataFrame(ability_location.dropna()).rename(columns={'event_contents': 'location'})

ability_location_df.to_csv(os.path.join(data_folder,'ability_location.csv'),index=False)

In [67]:
pd.DataFrame(aaa).head(1000)

,event_contents
0,[(1360) - TrainSCV]
1,"[(1021) - BuildSupplyDepot, Location: (135.0, 42.0, 49119)]"
2,[(480) - Stop]
3,[(1360) - TrainSCV]
4,[(1360) - TrainSCV]
5,"[(1021) - BuildSupplyDepot, Location: (28.0, 148.0, 49127)]"
6,"[(1022) - BuildRefinery, Target: CreepOnlyBlocker4x4 [024C0001], Location: (27.5, 154.5, 49120)]"
7,[(1360) - TrainSCV]
8,"[(1023) - BuildBarracks, Location: (135.5, 39.5, 49133)]"
9,[(1360) - TrainSCV]
